In [11]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import plotly.express as px

import re
import regex
import emoji
from collections import Counter
from wordcloud import WordCloud, STOPWORDS

# De txt a tabla

## Funciones

In [12]:
def IniciaConFechaYHora(s):
    # Ejemplo: '2/10/2018, 9:38 p. m. - ... '
    patron = '^([1-9]|1[0-9]|2[0-9]|3[0-1])(\/)([1-9]|1[0-2])(\/)(2[0-9][0-9][0-9]), ([0-9]|0[0-9]|1[0-2]):([0-9][0-9])\u202f(p.\u202fm.|a.\u202fm.) -'
    resultado = re.match(patron, s)  # Verificar si cada línea del txt hace match con el patrón de fecha y hora
    if resultado:
        return True
    return False

In [13]:
# Patron para encontrar a los miembros del grupo dentro del txt
def EncontrarMiembro(s):
    patrones = [
        '([\w]+):',                                    # Nombre
        '([\w]+[\s]+[\(]+[\w]+[\)]+):',      # Nombre (Apodo)
        '([\w]+[\s]+[\w]+):',                    # Nombre + Apellido
        '([\w]+[\s]+[\w]+[\s]+[\w]+):',    # Nombre 1 + Nombre 2 + Apellido
        '([+]\d{2} \d{3} \d{3} \d{4}):',     # Número de teléfono (MX)
        '([\w]+)[\u263a-\U0001f999]+:', # Nombre + Emoji            
    ]
    patron = '^' + '|'.join(patrones)     
    resultado = re.match(patron, s)  # Verificar si cada línea del txt hace match con el patrón de miembro
    if resultado:
        return True
    return False

In [14]:
# Separar las partes de cada línea del txt: Fecha, Hora, Miembro y Mensaje
def ObtenerPartes(linea):   
    # Ejemplo: '21/2/2021 11:27 a. m. - Sandro: Todos debemos aprender a analizar datos'
    splitLinea = linea.split(' - ') 
    FechaHora = splitLinea[0]                     # '21/2/2021 11:27 a. m.'
    splitFechaHora = FechaHora.split(' ')   
    Fecha = splitFechaHora[0]                    # '21/2/2021'
    Hora = ' '.join(splitFechaHora[1:])          # '11:27 a. m.'
    Mensaje = ' '.join(splitLinea[1:])             # 'Sandro: Todos debemos aprender a analizar datos'
    if EncontrarMiembro(Mensaje): 
        splitMensaje = Mensaje.split(': ')      
        Miembro = splitMensaje[0]               # 'Sandro' 
        Mensaje = ' '.join(splitMensaje[1:])    # 'Todos debemos aprender a analizar datos'
    else:
        Miembro = None
    return Fecha, Hora, Miembro, Mensaje

### Unit test

In [3]:

import unittest

class TestIniciaConFechaYHora(unittest.TestCase):
    def test_IniciaConFechaYHora(self):
        self.assertTrue(IniciaConFechaYHora("24/10/2018, 10:36 p. m. - Paola Daft: 😹😹😹"))
        self.assertFalse(IniciaConFechaYHora("Esto no comienza con una fecha y hora"))
        self.assertTrue(IniciaConFechaYHora("1/1/2000, 12:00 a. m. - Usuario: mensaje"))
        self.assertFalse(IniciaConFechaYHora("24/10/2018 - Paola Daft: 😹😹😹"))  # Falta la hora
        self.assertFalse(IniciaConFechaYHora("10:36 p. m. - Paola Daft: 😹😹😹"))  # Falta la fecha

suite = unittest.TestLoader().loadTestsFromTestCase(TestIniciaConFechaYHora)
unittest.TextTestRunner().run(suite)

.
----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

## Obtención de las partes del chat